<h2 align='center'>NLP Tutorial: Text Representation - Bag Of Words (BOW)</h2>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("spam.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.Category.value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [4]:
df['spam'] = df['Category'].apply(lambda x: 1 if x =='spam' else 0)

In [5]:
df.shape

(5572, 3)

In [6]:
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


<h3>Train test split</h3>

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Message, df.spam, test_size=0.2)

In [8]:
X_train.shape

(4457,)

In [9]:
X_test.shape

(1115,)

In [10]:
type(X_train)

pandas.core.series.Series

In [11]:
X_train[:4]

253                           What you doing?how are you?
845                              K.k:)when are you going?
96                      Watching telugu movie..wat abt u?
1214    Yeah, probably but not sure. Ilol let u know, ...
Name: Message, dtype: object

In [12]:
type(y_train)

pandas.core.series.Series

In [13]:
y_train[:4]

253     0
845     0
96      0
1214    0
Name: spam, dtype: int64

In [14]:
type(X_train.values)

numpy.ndarray

<h3>Create bag of words representation using CountVectorizer</h3>

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

X_train_cv = v.fit_transform(X_train.values)
X_train_cv

<4457x7803 sparse matrix of type '<class 'numpy.int64'>'
	with 59624 stored elements in Compressed Sparse Row format>

In [16]:
X_train_cv.toarray()[:2][0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [17]:
X_train_cv.shape

(4457, 7803)

In [18]:
v.get_feature_names_out()[1771]

'chex'

In [19]:
v.vocabulary_

{'what': 7533,
 'you': 7764,
 'doing': 2411,
 'how': 3559,
 'are': 1045,
 'when': 7539,
 'going': 3200,
 'watching': 7458,
 'telugu': 6796,
 'movie': 4651,
 'wat': 7453,
 'abt': 759,
 'yeah': 7735,
 'probably': 5463,
 'but': 1579,
 'not': 4866,
 'sure': 6664,
 'ilol': 3657,
 'let': 4131,
 'know': 4008,
 'personally': 5200,
 'wuldnt': 7693,
 'bother': 1450,
 'then': 6863,
 'again': 854,
 'if': 3641,
 'ur': 7250,
 'goin': 3198,
 'to': 6973,
 'mite': 4568,
 'as': 1083,
 'well': 7511,
 'congratulations': 1970,
 'ore': 5017,
 'mo': 4583,
 'owo': 5070,
 're': 5631,
 'wa': 7395,
 'enjoy': 2622,
 'it': 3802,
 'and': 957,
 'wish': 7597,
 'many': 4392,
 'happy': 3366,
 'moments': 4608,
 'fro': 3046,
 'wherever': 7544,
 'go': 3188,
 'by': 1592,
 'monday': 4611,
 'next': 4806,
 'week': 7496,
 'give': 3171,
 'me': 4447,
 'the': 6850,
 'full': 3066,
 'gist': 3169,
 'nice': 4810,
 'is': 3791,
 'working': 7650,
 'huh': 3583,
 'also': 922,
 'cannot': 1641,
 'only': 4976,
 'mistakes': 4566,
 'six': 6227

In [20]:
X_train_np = X_train_cv.toarray()
X_train_np[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [21]:
np.where(X_train_np[0]!=0)

(array([1045, 2411, 3559, 7533, 7764], dtype=int64),)

In [23]:
X_train_np[0][1771]

0

<h3>Train the naive bayes model</h3>

In [24]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_cv, y_train)

MultinomialNB()

In [25]:
X_test_cv = v.transform(X_test)

<h3>Evaluate Performance</h3>

In [28]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_cv)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       975
           1       0.96      0.94      0.95       140

    accuracy                           0.99      1115
   macro avg       0.97      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [27]:
emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]

emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1], dtype=int64)

<h3>Train the model using sklearn pipeline and reduce number of lines of code</h3>

In [32]:
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [33]:
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [34]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       975
           1       0.96      0.94      0.95       140

    accuracy                           0.99      1115
   macro avg       0.97      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115

